In [1]:
import pandas as pd
import geopandas as gpd

Script responsável por obter as áreas incrementais das estações fluviométricas, tomando por base a BHO V6 da ANA. Muitas bacias hidrográficas podem possuir mais de uma estação fluviométrica em sua área de contribuição. Diante disso, em um processo de regionalização de vazões dessas estações, é necessário determinar quais são as áreas de contribuição de cada uma das estações, podendo assim ter maior precisão no processo de regionalização para os trechos de drenagem.  

Partindo da base BHO ANA B6 (https://metadados.snirh.gov.br/geonetwork/srv/por/catalog.search#/metadata/32e309da-a8c1-443f-90ac-0cd79ce6a33d), foram selecionados os trechos correspondentes às estações fluviométricas. Esses trechos serviram de base para selecionar tudo que estava a montante da respectiva estação, estabelecendo assim as áreas incrementais. Entende-se por área incremental aquela área existente entre uma estação de montante e outra de jusante (se houver).

In [2]:
bho_goias = gpd.read_file('bho_2022_goias_ana_versaoteste.gpkg') #lendo o gpkg da BHO
trechos_estacoes = gpd.read_file('trechos_estacoes_v4.gpkg') #lendo trechos da BHO correspondentes às estações


In [3]:
def obter_areas_incrementais_bho(bho_goias,trechos_escolhidos):
    trechos_estacoes_sorted = trechos_escolhidos.sort_values(by='cobacia',ascending=False) #ordena os trechos otto (maior otto para menor otto = montante para jusante)
    montante_estacoes = pd.DataFrame(columns=['cobacia','cocursodag']) #cria um df vazio, para armazenar os trechos selecionados
    for i in range(len(trechos_estacoes_sorted)):
        filter_montante = (bho_goias['cobacia']>=trechos_estacoes_sorted['cobacia'].iloc[i]) & (bho_goias['cobacia'].str.startswith(trechos_estacoes_sorted['cocursodag'].iloc[i])) #filtro de montante
        trechos_montantes_estacoes = bho_goias[filter_montante] #seleciona os trechos 
        trechos_montantes_estacoes['cobacia_referencia'] = trechos_estacoes_sorted['cobacia'].iloc[i] #cria para toda a seleção uma coluna com o otto de referencia da respectiva seleção
        #montante_estacoes = pd.concat([trechos_montantes_estacoes,montante_estacoes])
        if any(trechos_montantes_estacoes['cobacia'].isin(montante_estacoes['cobacia'])): #se algum dos trechos já existir no df vazio (inicialmente vazio), faça:
            diferenca_trechos = trechos_montantes_estacoes[~(trechos_montantes_estacoes['cobacia'].isin(montante_estacoes['cobacia']))] #obtenha o que não existe 
            montante_estacoes = pd.concat([diferenca_trechos,montante_estacoes]) #una o que existe com o que ainda não existe(diferença) no df vazio (montante_estacoes)
            
        else: #caso nenhum trecho esteja no df vazio,faça:
            montante_estacoes = pd.concat([trechos_montantes_estacoes,montante_estacoes]) # una os trechos ao df vazio (montante_estacoes)
    return montante_estacoes.dissolve(by='cobacia_referencia',as_index=False) #retorne o gpkg dissolvido para cada área de contribuição
        


In [4]:
teste = obter_areas_incrementais_bho(bho_goias, trechos_estacoes)
teste

c:\Users\matheus.cgomes\Documents\Notebooks\limpeza_durhs\venv\Lib\site-packages\geopandas\geodataframe.py:1543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
c:\Users\matheus.cgomes\Documents\Notebooks\limpeza_durhs\venv\Lib\site-packages\geopandas\geodataframe.py:1543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
c:\Users\matheus.cgomes\Documents\Notebooks\limpeza_durhs\venv\Lib\site-packages\geopandas\geodataframe.py:1543: SettingWithCopyWa

,cobacia_referencia,geometry,dra_pk,idbacia,cotrecho,cocursodag,cobacia,nuareacont,nuordemcda,nunivotto1,nunivotto2,nunivotto3,nunivotto4,nunivotto5,nunivotto6,nunivotto,dsversao
0,6658399,"POLYGON ((-49.35924 -13.60114, -49.35916 -13.6...",113.0,113.0,5366316.0,6658,66585573,2.730564,3.0,6,66,665,6658,66585,665855,8.0,BHO bho_v_06_02_04 de 2022-10-19
1,6662933,"POLYGON ((-46.62751 -13.03813, -46.62762 -13.0...",709.0,709.0,4413872.0,666294,66629455,3.452804,5.0,6,66,666,6662,66629,666294,8.0,BHO bho_v_06_02_04 de 2022-10-19
2,66635411,"POLYGON ((-47.72848 -13.39525, -47.72865 -13.3...",9938.0,9938.0,741746.0,6663544,666354493,26.431361,5.0,6,66,666,6663,66635,666354,9.0,BHO bho_v_06_02_04 de 2022-10-19
3,6663551,"POLYGON ((-47.50766 -13.48615, -47.50780 -13.4...",1189.0,1189.0,2431244.0,6663726,6663726,11.336085,5.0,6,66,666,6663,66637,666372,7.0,BHO bho_v_06_02_04 de 2022-10-19
4,666411,"POLYGON ((-46.77157 -13.60978, -46.77171 -13.6...",5498.0,5498.0,1441563.0,6664424,66644249,19.902130,7.0,6,66,666,6664,66644,666442,8.0,BHO bho_v_06_02_04 de 2022-10-19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89,8699211111,"POLYGON ((-48.34054 -18.36927, -48.34068 -18.3...",382.0,382.0,2427514.0,86992296,86992296,7.883045,5.0,8,86,869,8699,86992,869922,8.0,BHO bho_v_06_02_04 de 2022-10-19
90,86999111,"POLYGON ((-47.78875 -18.22172, -47.78882 -18.2...",2041.0,2041.0,5369737.0,869993199522,869993199522,3.661875,4.0,8,86,869,8699,86999,869993,12.0,BHO bho_v_06_02_04 de 2022-10-19
91,869992111,"POLYGON ((-47.60153 -18.13397, -47.60159 -18.1...",6051.0,6051.0,4712752.0,869992414,869992414,4.357337,5.0,8,86,869,8699,86999,869992,9.0,BHO bho_v_06_02_04 de 2022-10-19
92,869993535,"MULTIPOLYGON (((-47.64829 -17.30592, -47.64837...",6321.0,6321.0,3476081.0,86999452,869994523,8.470644,4.0,8,86,869,8699,86999,869994,9.0,BHO bho_v_06_02_04 de 2022-10-19


In [5]:
teste.to_file('teste_incremental_estacoes6.gpkg')